In [0]:
import pandas as pd
train_df = pd.read_csv("poem_train.csv", encoding="utf-8")
train_df

In [0]:
test_df = pd.read_csv("poem_test.csv", encoding="utf-8")
test_df

In [0]:
trans = {"李白":0, "杜甫":1, "白居易":2}
y_train = train_df["作者"].replace(trans)
y_test = test_df["作者"].replace(trans)
y_test

In [10]:
from jieba import cut
s = "平林漠漠煙如織，寒山一帶傷心碧。\r\n暝色入高樓，有人樓上愁。"
s = s.replace("\r", "").replace("\n", "")
" ".join(cut(s))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.779 seconds.
Prefix dict has been built succesfully.


'平林 漠漠 煙如織 ， 寒山 一帶 傷心 碧 。 暝 色入 高樓 ， 有人 樓上 愁 。'

In [14]:
# !!!!!!! apply
def poemcut(s):
    s = s.replace("\r", "").replace("\n", "")
    return " ".join(cut(s))
x_train = train_df["內容"].apply(poemcut)
x_test = test_df["內容"].apply(poemcut)
x_test

0      日照 香爐生 紫煙 ， 遙看 瀑布 掛 前川 。 飛流 直下 三千尺 ， 疑是 銀河 落九天 。
1     朝辭 白帝 彩雲間 ， 千裡 江陵 一日 還 。 兩岸 猿聲 啼 不住 ， 輕舟 已過 萬 ...
2      李白 乘舟 將欲行 ， 忽聞 岸上 踏歌 聲 。 桃花潭水 深 千尺 ， 不及 汪倫送 我情 。
3       故人 西辭黃鶴樓 ， 煙花 三月 下揚州 。 孤帆 遠影 碧空 儘 ， 唯見長 江天 際流 。
4                 危樓 高 百尺 ， 手可摘 星辰 。 不敢 高聲語 ， 恐驚 天上 人 。
5                  床前 明月光 ， 疑是 地上 霜 。 舉頭 望明月 ， 低頭思 故鄉 。
6     天門 中斷 楚江 開 ， 碧水 東流 至此 回 。 兩岸 青山 相對 出 ， 孤帆 一片 日...
7               眾鳥 高 飛 儘 ， 孤雲獨 去 閒 。 相看 兩不厭 ， 隻 有 敬亭山 。
8     鳳凰 台上 鳳凰遊 ， 鳳去 台空江 自流 。 吳宮 花草 埋 幽徑 ， 晉代 衣冠 成古丘...
9     渡遠 荊門外 ， 來 從 楚國遊 。 山 隨 平野 儘 ， 江入 大荒 流 。 月 下 飛天...
10    百川 日東流 ， 客去 亦 不息 。 我 生苦 漂 蕩 ， 何時 有 終極 。 讚 公 釋 ...
11    細泉 兼 輕冰 ， 沮洳 棧道 濕 。 不辭 辛苦 行 ， 迫此 短景急 。 石門 雪雲隘 ...
12    首路 栗亭 西 ， 尚想 鳳凰村 。 季冬 攜 童稚 ， 辛苦 赴 蜀門 。 南登 木皮 嶺...
13    落日 在 簾 鉤 ， 溪邊 春事幽 。 芳菲 緣岸 圃 ， 樵 爨 倚灘 舟 。 啅 雀 爭...
14    竇侍 禦 ， 驥 之子 ， 鳳之雛 。 年 未 三十 忠義俱 ， 骨鯁 絕代 無 。 炯 如...
15    百草 競春華 ， 麗春應 最勝 。 少須 好 顏色 ， 多漫枝條 剩 。 紛紛 桃李 枝 ，...
16    何年 顧虎頭 ， 滿壁畫 瀛州 。 赤 日 石林 氣 ， 青天 江海 流 。 錫飛常 近鶴 ...
17    野寺 隱喬木 ， 山僧 高下 居 。 石門 日色異 ， 絳 氣橫 扶疏 。 窈窕 入

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_vec = vec.fit_transform(x_train)

In [22]:
# 到底有多少個詞
len(vec.vocabulary_)

52294

In [0]:
print(x_train_vec)

In [0]:
x_test_vec = vec.transform(x_test)

In [25]:
# https://scikit-learn.org/stable/modules/naive_bayes.html#naive-bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_vec, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_vec)
accuracy_score(pre, y_test)

0.8

In [30]:
from sklearn.metrics import confusion_matrix
import pandas as pd
n = ["李白", "杜甫", "白居易"]
pd.DataFrame(confusion_matrix(y_test, pre), 
             columns=n,
             index=n)

,李白,杜甫,白居易
李白,8,1,1
杜甫,1,8,1
白居易,1,1,8


In [35]:
trans = ["李白", "杜甫", "白居易"]
p = input("寫首詩吧!")
v = vec.transform([poemcut(p)])
i = clf.predict(v)[0]
print("應該是誰寫的?", trans[i])

寫首詩吧!君不見黃河之水天上來，奔流到海不復回。 君不見高堂明鏡悲白髮，朝如青絲暮成雪。 人生得意須盡歡，莫使金樽空對月。 天生我材必有用，[2]千金散盡還復來。 烹羊宰牛且爲樂，會須一飲三百杯。
應該是誰寫的? 李白
